In [2]:
# 크기 변경
import numpy as np
import cv2

def scaling(img, size):
    dst = np.zeros(size[::-1], img.dtype)
    ratioY, ratioX = np.divide(size[::-1], img.shape[:2])
    y = np.arange(0, img.shape[0], 1)
    x = np.arange(0, img.shape[1], 1)
    y, x = np.meshgrid(y,x)
    i, j = np.int32(y*ratioY), np.int32(x*ratioX)
    dst[i,j] = img[y,x]
    return dst


def bilinear_value(img,pt):
    x,y = np.int32(pt)
    if x>=img.shape[1]-1:x=x-1
    if y>=img.shape[0]-1:y=y-1
    P1, P2, P3, P4 = np.float32(img[y:y+2, x:x+2].flat)
    alpha, beta = pt[1]-y, pt[0]-x
    M1 = P1 + alpha*(P3 - P1)
    M2 = P2 + alpha*(P4 - P2)
    P = M1 + beta*(M2 - M1)
    return np.clip(P,0,255).astype(np.uint8)

def scaling_bilinear(img, size):
    ratioY, ratioX = np.divide(size[::-1], img.shape[:2])
    dst = [[bilinear_value(img, (j/ratioX, i/ratioY)) for j in range(size[0])] for i in range(size[1])]
    return np.array(dst, img.dtype)

image = cv2.imread(r"img/img.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상파일 읽기 오류")

size = (image.shape[1]*2, image.shape[0]*2)

dst1 = scaling(image,size)
dst2 = scaling_bilinear(image,size)
cv2.imshow("image", image)
cv2.imshow("scaling - nearest", dst1)
cv2.imshow("scaling - bilinear", dst2)
cv2.waitKey(0)

-1

In [ ]:
import numpy as np
import cv2

def contain(p,shape):
    return 0 <= p[0] < shape[1] and 0 <= p[1] < shape[0]

def translate(img,pt):
    dst = np.zeros(img.shape, img.dtype)
    for i in range(img.shape[0]):
        for j in range(img.shape[0]):
            x,y = np.subtract((j,i),pt)
            if contain((x,y), img.shape):
                dst[i,j] = img[y,x]
    return dst

image = cv2.imread(r"img/img.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상파일 읽기 오류") 

dst1 = translate(image,(30,30))
dst2 = translate(image,(-70,70))

cv2.imshow("image", image)
cv2.imshow("translate +30,+30", dst1)
cv2.imshow("translate -70,+70", dst2)   
cv2.waitKey(0)